## 네이버 뉴스 크롤러

- 키워드 입력시, 키워드에 해당하는 뉴스기사 크롤링
- 페이지 설정 가능
- 크롤링 한 뉴스기사 모아서, 단어 빈도수 Counting
- 상위 30개 출력

---

### 크롤러_V1

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import re
from konlpy.tag import Twitter
from konlpy.tag import Kkma
from collections import Counter
import jpype
import pandas


si_dong = input("keyword : ")
page = int(input("page (0 ~ ): ")) -1

ua = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.87 Safari/537.36'}

urls = []
for page in range(1, page + 1):
    url = "https://search.naver.com/search.naver?&where=news&query={}&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:dd,p:all,a:all&mynews=0&start={}1&refresh_start=0".format(si_dong,page)

    response = requests.get(url, headers = ua)
    html = BeautifulSoup(response.text, 'html.parser')

    li_list = html.select("ul.type01 li")

    for li in li_list:
        ul = li.select_one('dl dd a').get('href')
        if ul == '#':
            pass
        else:
            urls.append(ul)
    time.sleep(1)

def refine_text(text):
    text = text.replace("\n","").replace("\t","").replace("\r","").replace('// flash 오류를 우회하기 위한 함수 추가','')
    text = text.replace('function _flash_removeCallback() {}','').replace('“','').replace("”",'')
    text = text.replace('‘','').replace("’",'').replace('"','').replace("'",'').replace('  ','')
    text = text.replace('인턴기자','').replace('기자','').replace('뉴스','')
    text = text.strip()
    
    return text

news = ''

for ul in urls:
    response = requests.get(ul, headers = ua)
    html = BeautifulSoup(response.text, 'html.parser')

    html.select_one("#content div.end_ct_area")
    
    try:
        body = refine_text(html.select_one("#articleBodyContents").text).split('@')[0]
    except:
        continue
    body = body.split('▶')[0]
    body = body.split('▲')[0]
    body = body.split('◇')[0]
    body = re.sub(r'\(.*?\)',"",body)
    body = re.sub(r'\[.*?\]',"",body)
    body = re.sub(r'\【.*?\】',"",body)
    body = body[:-15].strip().replace('(','')
    news += body

kkma = Kkma()
twitter = Twitter()

nouns = twitter.nouns(news)

wdlist = []
for i in range(0,len(nouns)):
    if (len(nouns[i])) >= 2 and nouns[i] != si_dong and nouns[i] != si_dong[:-1]:
        wdlist.append(nouns[i])

wc = Counter(wdlist)       # 각 단어별 빈도수 측정

wc.most_common(30)


keyword : 부동산
page (0 ~ ): 5


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


[('분양', 32),
 ('상한', 29),
 ('주택', 25),
 ('사업', 20),
 ('개발', 19),
 ('시행', 18),
 ('국토', 17),
 ('연구원', 17),
 ('분석', 16),
 ('서울', 16),
 ('하락', 15),
 ('만주', 15),
 ('금리', 14),
 ('가격', 13),
 ('정부', 13),
 ('시장', 13),
 ('집값', 12),
 ('조선', 11),
 ('지역', 11),
 ('대박', 11),
 ('재건축', 10),
 ('투자', 10),
 ('증가', 10),
 ('자산', 10),
 ('규제', 10),
 ('택배', 10),
 ('수도권', 9),
 ('확대', 9),
 ('효과', 9),
 ('아파트', 9)]